In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [156]:
from dblm.core.modeling import gpt2, bert
from transformers import EncoderDecoderModel
from dblm.utils.seeding import seed
from torch.nn import CrossEntropyLoss
import torch
import math
from itertools import product
import gc
from tqdm.notebook import tqdm

def generate_config(*, nvars, nvals, decoder_n_layer, tool_n_layer, seed, hidden_dim, attention_mode, initializer_range, remove_ln):
    data_config = {
        "nvars": nvars,
        "nvals": nvals,
        "decoder_n_layer": decoder_n_layer,
        "tool_n_layer": tool_n_layer,
        "seed": seed,
        "attention_mode": attention_mode,
        "remove_ln": remove_ln
    }
    data_config["seq_len"] = 2 * data_config["nvars"]
    data_config["encoder_vocab_size"] = data_config["nvals"] + 1
    data_config["encoder_pad_token_id"] = 0
    data_config["decoder_vocab_size"] = data_config["nvals"] + 1 + data_config["nvars"]
    data_config["decoder_bos_token_id"] = data_config["nvals"]
    data_config["initializer_range"] = initializer_range
    data_config["hidden_dim"] = hidden_dim
    return data_config

def build_model(data_config, decoder_only=False):
    if not decoder_only:

        seed(data_config["seed"])
        encoder_config = bert.BertConfig(
            num_hidden_layers=0,
            vocab_size=data_config["encoder_vocab_size"],
            max_position_embeddings=data_config["nvars"] * data_config["nvals"],
            pad_token_id=data_config["encoder_pad_token_id"],
            initializer_range=data_config["initializer_range"],
            intermediate_size=data_config["hidden_dim"]*4,
            hidden_size=data_config["hidden_dim"],
        )
        decoder_config = gpt2.GPT2Config(
            num_hidden_layers=data_config["decoder_n_layer"],
            vocab_size=data_config["decoder_vocab_size"],
            max_position_embeddings=data_config["seq_len"],
            bos_token_id=data_config["decoder_bos_token_id"],
            add_cross_attention=True,
            is_decoder=True,
            initializer_range=data_config["initializer_range"],
            n_embd=data_config["hidden_dim"],
            n_inner=data_config["hidden_dim"]*4,
            remove_ln_cross_attn=data_config["remove_ln"],
        )
        tool_config = gpt2.GPT2Config(
            num_hidden_layers=data_config["tool_n_layer"],
            vocab_size=data_config["decoder_vocab_size"],
            max_position_embeddings=data_config["seq_len"],
            bos_token_id=data_config["decoder_bos_token_id"],
            add_cross_attention=False,
            is_decoder=True,
            initializer_range=data_config["initializer_range"],
            num_labels=data_config["nvars"] * data_config["nvals"],
            n_embd=data_config["hidden_dim"],
            n_inner=data_config["hidden_dim"]*4,
        )
        encoder = bert.BertModel(encoder_config)
        decoder = gpt2.GPT2LMHeadModel(decoder_config)
        tool = gpt2.GPT2ForTokenClassification(tool_config).to("cuda")
        model = EncoderDecoderModel(encoder=encoder, decoder=decoder).to("cuda")
        model.data_config = data_config
        
        mask1 = (((torch.arange(data_config["seq_len"])+1)%2)[None,:,None] * (torch.arange(decoder.config.vocab_size) <= data_config["nvals"])[None,None,:]).to(torch.bool).to("cuda")
        mask2 = (((torch.arange(data_config["seq_len"]))%2)[None,:,None] * (torch.arange(decoder.config.vocab_size) >= data_config["nvals"])[None,None,:]).to(torch.bool).to("cuda")
        mask = mask1 | mask2
        loss_fn = CrossEntropyLoss(reduction="none")
        def log_prob(logits, labels):
            batch, seq, vocab = logits.size()
            logits = logits.masked_fill(mask.expand_as(logits), -math.inf)
            lprob = -loss_fn(logits.reshape(batch * seq, -1), labels.reshape(-1)).reshape(batch, seq)
            return lprob
        model.log_prob_fn = log_prob
        model.tool = tool
        model.eval()
        return model
    else:
        decoder_config = gpt2.GPT2Config(
            num_hidden_layers=data_config["decoder_n_layer"],
            vocab_size=data_config["decoder_vocab_size"],
            max_position_embeddings=data_config["seq_len"],
            bos_token_id=data_config["decoder_bos_token_id"],
            is_decoder=True,
            initializer_range=data_config["initializer_range"],
            n_embd=data_config["hidden_dim"],
            n_inner=data_config["hidden_dim"]*4,
            remove_ln_cross_attn=data_config["remove_ln"],
        )
        model = gpt2.GPT2LMHeadModel(decoder_config).to("cuda")
        model.eval()
        
        mask1 = (((torch.arange(data_config["seq_len"])+1)%2)[None,:,None] * (torch.arange(model.config.vocab_size) <= data_config["nvals"])[None,None,:]).to(torch.bool).to("cuda")
        mask2 = (((torch.arange(data_config["seq_len"]))%2)[None,:,None] * (torch.arange(model.config.vocab_size) >= data_config["nvals"])[None,None,:]).to(torch.bool).to("cuda")
        mask = mask1 | mask2
        loss_fn = CrossEntropyLoss(reduction="none")
        def log_prob(logits, labels):
            batch, seq, vocab = logits.size()
            logits = logits.masked_fill(mask.expand_as(logits), -math.inf)
            lprob = -loss_fn(logits.reshape(batch * seq, -1), labels.reshape(-1)).reshape(batch, seq)
            return lprob
        model.log_prob_fn = log_prob
        return model

def log_prob_nested(model, labels):
    if isinstance(model, gpt2.GPT2LMHeadModel):
        decoder_input_ids = torch.cat([labels.new_ones(labels.size(0), 1).fill_(model.config.bos_token_id), labels[:, :-1]], dim=1)
        logits = model(input_ids=decoder_input_ids).logits
        return model.log_prob_fn(logits, labels)
    else:
        batch_size, seq_len = labels.size()
        decoder_input_ids = torch.cat([labels.new_ones(labels.size(0), 1).fill_(model.decoder.config.bos_token_id), labels[:, :-1]], dim=1)
        tool_output = model.tool(input_ids=decoder_input_ids).logits.reshape(batch_size, seq_len, model.data_config["nvars"], model.data_config["nvals"]).log_softmax(-1).reshape(batch_size, seq_len, -1)
        all_input_ids = torch.tensor(list(range(model.data_config["nvals"])) * model.data_config["nvars"]).to("cuda").unsqueeze(0)
        all_attention_mask = torch.tensor([1] * model.data_config["nvals"] * model.data_config["nvars"]).to("cuda").unsqueeze(0)
        all_position_ids = torch.tensor([i for i in range(model.data_config["nvars"]) for j in range(model.data_config["nvals"])]).to("cuda").unsqueeze(0)
        nested_decoder_output = model(input_ids=all_input_ids, attention_mask=all_attention_mask, position_ids=all_position_ids, decoder_input_ids=decoder_input_ids, labels=labels, decoder_encoder_log_marginals=tool_output, decoder_encoder_attention_mode=model.data_config["attention_mode"])
        return model.log_prob_fn(nested_decoder_output.logits, labels)

def cross_entropy_with_grad(m1, m2):
    cfg = m1.data_config
    nvars = cfg["nvars"]
    nvals = cfg["nvals"]
    H = 0
    batch_size = 16
    increase = True
    acc = 0
    batch = []
    prev_mem = None
    
        
    for label_ids in tqdm(product(*sum(([list(range(cfg["nvals"]+1, cfg["nvals"]+1+cfg["nvars"])), list(range(cfg["nvals"]))] for i in range(cfg["nvars"])), [])), total=((cfg["nvals"] * cfg["nvars"]) ** cfg["nvars"]), leave=False):
        batch.append(label_ids)
        acc += 1
        if acc % batch_size == 0:
            if len(batch) == 0:
                continue
            log_prob1 = log_prob_nested(m1, torch.tensor(batch).to("cuda")).sum(dim=-1)
            log_prob2 = log_prob_nested(m2, torch.tensor(batch).to("cuda")).sum(dim=-1)
            H -= (log_prob2 * log_prob1.exp()).sum()
            acc = 0
            batch = []
            increase = increase and (prev_mem is None or ((torch.cuda.max_memory_allocated("cuda:0") * 3 - prev_mem * 2) < 20000000000))
            if increase:
                batch_size = int(batch_size * 1.1)
            prev_mem = torch.cuda.max_memory_allocated("cuda:0")
    if len(batch) > 0:
        log_prob1 = log_prob_nested(m1, torch.tensor(batch).to("cuda")).sum(dim=-1)
        log_prob2 = log_prob_nested(m2, torch.tensor(batch).to("cuda")).sum(dim=-1)
        H -= (log_prob2 * log_prob1.exp()).sum()
    return H

@torch.no_grad()
def cross_entropy(m1, m2):
    cfg = m1.data_config
    nvars = cfg["nvars"]
    nvals = cfg["nvals"]
    H = 0
    batch_size = 16
    increase = True
    acc = 0
    batch = []
    prev_mem = None
    
        
    for label_ids in tqdm(product(*sum(([list(range(cfg["nvals"]+1, cfg["nvals"]+1+cfg["nvars"])), list(range(cfg["nvals"]))] for i in range(cfg["nvars"])), [])), total=((cfg["nvals"] * cfg["nvars"]) ** cfg["nvars"]), leave=False):
        batch.append(label_ids)
        acc += 1
        if acc % batch_size == 0:
            if len(batch) == 0:
                continue
            log_prob1 = log_prob_nested(m1, torch.tensor(batch).to("cuda")).sum(dim=-1)
            log_prob2 = log_prob_nested(m2, torch.tensor(batch).to("cuda")).sum(dim=-1)
            H -= (log_prob2 * log_prob1.exp()).sum().item()
            acc = 0
            batch = []
            increase = increase and (prev_mem is None or ((torch.cuda.max_memory_allocated("cuda:0") * 3 - prev_mem * 2) < 20000000000))
            if increase:
                batch_size = int(batch_size * 1.1)
            prev_mem = torch.cuda.max_memory_allocated("cuda:0")
    if len(batch) > 0:
        log_prob1 = log_prob_nested(m1, torch.tensor(batch).to("cuda")).sum(dim=-1)
        log_prob2 = log_prob_nested(m2, torch.tensor(batch).to("cuda")).sum(dim=-1)
        H -= (log_prob2 * log_prob1.exp()).sum().item()
    return H

@torch.no_grad()
def cross_entropy_tools(m1, m2):
    cfg = m1.data_config
    nvars = cfg["nvars"]
    nvals = cfg["nvals"]
    H = 0
    batch_size = 16
    increase = True
    acc = 0
    batch = []
    prev_mem = None
    
        
    for label_ids in tqdm(product(*sum(([list(range(cfg["nvals"]+1, cfg["nvals"]+1+cfg["nvars"])), list(range(cfg["nvals"]))] for i in range(cfg["nvars"])), [])), total=((cfg["nvals"] * cfg["nvars"]) ** cfg["nvars"]), leave=False):
        batch.append(label_ids)
        acc += 1
        if acc % batch_size == 0:
            if len(batch) == 0:
                continue
            batch_size, seq_len = len(batch), len(batch[0])
            log_prob1 = m1.tool(input_ids=torch.tensor(batch).to("cuda")).logits.reshape(batch_size, seq_len, nvars, nvals).log_softmax(-1).reshape(batch_size, seq_len, -1)
            log_prob2 = m2.tool(input_ids=torch.tensor(batch).to("cuda")).logits.reshape(batch_size, seq_len, nvars, nvals).log_softmax(-1).reshape(batch_size, seq_len, -1)
            log_weight = log_prob_nested(m1, torch.tensor(batch).to("cuda")).sum(dim=-1)
            H -= ((log_prob2 * log_prob1.exp()).sum(-1).mean(-1) * log_weight.exp()).sum().item()
            acc = 0
            batch = []
            increase = increase and (prev_mem is None or ((torch.cuda.max_memory_allocated("cuda:0") * 3 - prev_mem * 2) < 20000000000))
            if increase:
                batch_size = int(batch_size * 1.1)
            prev_mem = torch.cuda.max_memory_allocated("cuda:0")
    if len(batch) > 0:
        batch_size, seq_len = len(batch), len(batch[0])
        log_prob1 = m1.tool(input_ids=torch.tensor(batch).to("cuda")).logits.reshape(batch_size, seq_len, nvars, nvals).log_softmax(-1).reshape(batch_size, seq_len, -1)
        log_prob2 = m2.tool(input_ids=torch.tensor(batch).to("cuda")).logits.reshape(batch_size, seq_len, nvars, nvals).log_softmax(-1).reshape(batch_size, seq_len, -1)
        log_weight = log_prob_nested(m1, torch.tensor(batch).to("cuda")).sum(dim=-1)
        H -= ((log_prob2 * log_prob1.exp()).sum(-1).mean(-1) * log_weight.exp()).sum().item()
    return H

def cross_entropy_estimate(m1, m2, n=1024, rng=None):
    if rng is not None:
        seed(rng)
    cfg = m1.data_config
    nvars = cfg["nvars"]
    nvals = cfg["nvals"]
    H = 0
    batch_size = 16
    increase = True
    acc = 0
    batch = []
    prev_mem = None

    label_id_samples = []
    total = 1
    for i in range(nvars):
        # uniform sample var index
        label_id_samples.append(torch.randint(nvals+1, nvals+1+nvars, (n,)))
        # uniform sample val index
        label_id_samples.append(torch.randint(0, nvals, (n,)))
        total *= nvals * nvars
    label_id_samples = torch.stack(label_id_samples, dim=1)

    for label_ids_idx in tqdm(range(len(label_id_samples)), leave=False):
        batch.append(label_ids_idx)
        acc += 1
        if acc % batch_size == 0:
            if len(batch) == 0:
                continue
            log_prob1 = log_prob_nested(m1, label_id_samples[batch].to("cuda")).sum(dim=-1)
            log_prob2 = log_prob_nested(m2, label_id_samples[batch].to("cuda")).sum(dim=-1)
            H -= (log_prob2 * log_prob1.exp()).sum()
            acc = 0
            batch = []
            increase = increase and (prev_mem is None or ((torch.cuda.max_memory_allocated("cuda:0") * 3 - prev_mem * 2) < 20000000000))
            if increase:
                batch_size = int(batch_size * 2)
            prev_mem = torch.cuda.max_memory_allocated("cuda:0")
    if len(batch) > 0:
        log_prob1 = log_prob_nested(m1, label_id_samples[batch].to("cuda")).sum(dim=-1)
        log_prob2 = log_prob_nested(m2, label_id_samples[batch].to("cuda")).sum(dim=-1)
        H -= (log_prob2 * log_prob1.exp()).sum()
    return H * total / n


In [30]:
import torch
print(torch.cuda.max_memory_allocated("cuda:0"))
model1 = build_model(generate_config(nvars=3, nvals=4, decoder_n_layer=16, tool_n_layer=8, seed=8945, hidden_dim=24, attention_mode="post_scale:albo", initializer_range=0.08, remove_ln=True))
model2 = build_model(generate_config(nvars=3, nvals=4, decoder_n_layer=16, tool_n_layer=8, seed=1245, hidden_dim=24, attention_mode="post_scale:albo", initializer_range=0.08, remove_ln=True))
with torch.no_grad():
    print("Hp(q)", cross_entropy(model1, model2))
    print("H(p)", cross_entropy(model1, model1))
model1.decoder

6021927424


  0%|          | 0/1728 [00:00<?, ?it/s]

Hp(q) 8.198939606547356


  0%|          | 0/1728 [00:00<?, ?it/s]

H(p) 7.09758809953928


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(8, 24)
    (wpe): Embedding(6, 24)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-15): 16 x GPT2Block(
        (ln_1): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
        (crossattention): GPT2Attention(
          (c_attn): Conv1D()
          (q_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_cross_attn): Identity()
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
 

In [31]:
model2.zero_grad()
model2.tool.zero_grad()
M=40
for i in range(M):
    ce = cross_entropy_estimate(model1, model2)
    (ce/M).backward()
print(sum(p.grad.norm()**2 for p in model2.parameters() if p.grad is not None) ** 0.5)
print(sum(p.grad.norm()**2 for p in model2.tool.parameters() if p.grad is not None) ** 0.5)
model2.zero_grad()
model2.tool.zero_grad()

  0%|          | 0/1024 [00:00<?, ?it/s]

  0%|          | 0/1024 [00:00<?, ?it/s]

  0%|          | 0/1024 [00:00<?, ?it/s]

  0%|          | 0/1024 [00:00<?, ?it/s]

  0%|          | 0/1024 [00:00<?, ?it/s]

  0%|          | 0/1024 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [32]:
params = list(model2.parameters()) + list(model2.tool.parameters())
optimizer = torch.optim.Adam(params, lr=1e-4)
iter = 0

In [47]:
model2.zero_grad()
model2.tool.zero_grad()
import itertools
grad_acc_steps = 10
steps = 50
for i in range(steps):
    cumloss = 0
    for i in range(grad_acc_steps):
        loss = cross_entropy_estimate(model1, model2) / grad_acc_steps
        # loss = cross_entropy_with_grad(model1, model2) / grad_acc_steps
        loss.backward()
        cumloss += loss.item()
    gnorm = sum(p.grad.norm()**2 for p in itertools.chain(model2.parameters(), model2.tool.parameters()) if p.grad is not None) ** 0.5
    optimizer.step()
    optimizer.zero_grad()
    iter += 1
    print(iter, cumloss, gnorm.item())

  0%|          | 0/1024 [00:00<?, ?it/s]

  0%|          | 0/1024 [00:00<?, ?it/s]

  0%|          | 0/1024 [00:00<?, ?it/s]

  0%|          | 0/1024 [00:00<?, ?it/s]

  0%|          | 0/1024 [00:00<?, ?it/s]

...

  0%|          | 0/1024 [00:00<?, ?it/s]

  0%|          | 0/1024 [00:00<?, ?it/s]

  0%|          | 0/1024 [00:00<?, ?it/s]

  0%|          | 0/1024 [00:00<?, ?it/s]

85 7.224312245845795 1.6922144889831543


In [48]:
model3 = build_model(generate_config(nvars=3, nvals=4, decoder_n_layer=16, tool_n_layer=8, seed=1245, hidden_dim=24, attention_mode="post_scale:albo", initializer_range=0.08, remove_ln=True))

with torch.no_grad():
    print("Hp(q0)", cross_entropy(model1, model3))
    print("Hp(q)", cross_entropy(model1, model2))
    print("H(p)", cross_entropy(model1, model1))
    print("Tool Hp(q0)", cross_entropy_tools(model1, model3))
    print("Tool Hp(q)", cross_entropy_tools(model1, model2))
    print("Tool H(p)", cross_entropy_tools(model1, model1))

  0%|          | 0/1728 [00:00<?, ?it/s]

Hp(q0) 8.198939606547356


  0%|          | 0/1728 [00:00<?, ?it/s]

Hp(q) 7.1022443771362305


  0%|          | 0/1728 [00:00<?, ?it/s]

...

Tool Hp(q0) 4.436533633619547


  0%|          | 0/1728 [00:00<?, ?it/s]

Tool Hp(q) 4.364886183291674


  0%|          | 0/1728 [00:00<?, ?it/s]

Tool H(p) 3.9319173097610474


In [ ]:
model1.decoder

In [ ]:
with torch.no_grad():
    input_ids = torch.tensor([0] * model.data_config["nvars"]).to("cuda").unsqueeze(0)
    attention_mask = torch.tensor([1] * model.data_config["nvars"]).to("cuda").unsqueeze(0)
    label_ids = sum(([i+model.data_config["nvals"]+1, i%model.data_config["nvals"]] for i in range(model.data_config["nvars"])), [])
    decoder_input_ids = torch.tensor([model.data_config["nvals"]] + label_ids[:-1]).to("cuda").unsqueeze(0)
    labels = torch.tensor(label_ids).to("cuda").unsqueeze(0)
    batch_size, seq_len = decoder_input_ids.size()
    # VAE
    vae_decoder_output = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, labels=labels)
    # Nested
    tool_output = tool(input_ids=decoder_input_ids).logits.reshape(batch_size, seq_len, model.data_config["nvars"], model.data_config["nvals"]).log_softmax(-1).reshape(batch_size, seq_len, -1)
    all_input_ids = torch.tensor(list(range(model.data_config["nvals"])) * model.data_config["nvars"]).to("cuda").unsqueeze(0)
    all_attention_mask = torch.tensor([1] * model.data_config["nvals"] * model.data_config["nvars"]).to("cuda").unsqueeze(0)
    all_position_ids = torch.tensor([i for i in range(model.data_config["nvars"]) for j in range(model.data_config["nvals"])]).to("cuda").unsqueeze(0)
    nested_decoder_output = model(input_ids=all_input_ids, attention_mask=all_attention_mask, position_ids=all_position_ids, decoder_input_ids=decoder_input_ids, labels=labels, decoder_encoder_log_marginals=tool_output, decoder_encoder_attention_mode=model.data_config["attention_mode"])
    print(labels)
    print("vae", log_prob_fn(vae_decoder_output.logits, labels).exp())
    print("nested", log_prob_fn(nested_decoder_output.logits, labels).exp())


In [ ]:
log_prob_nested(model, labels.repeat(10,1))

# Experiment 1

In [123]:
model1 = build_model(generate_config(nvars=3, nvals=4, decoder_n_layer=16, tool_n_layer=8, seed=8945, hidden_dim=24, attention_mode="post_scale:mult", initializer_range=0.08, remove_ln=True))
model2 = build_model(generate_config(nvars=3, nvals=4, decoder_n_layer=16, tool_n_layer=8, seed=1349, hidden_dim=24, attention_mode="post_scale:albo", initializer_range=0.08, remove_ln=True))
model3 = build_model(generate_config(nvars=3, nvals=4, decoder_n_layer=16, tool_n_layer=8, seed=1349, hidden_dim=24, attention_mode="post_scale:mult", initializer_range=0.08, remove_ln=True))
model2.tool = model1.tool
model3.tool = model1.tool
with torch.no_grad():
    print("Hp(qalbo)", cross_entropy(model1, model2))
    print("Hp(qmult)", cross_entropy(model1, model3))
    print("H(p)", cross_entropy(model1, model1))

  0%|          | 0/1728 [00:00<?, ?it/s]

Hp(qalbo) 8.112020283937454


  0%|          | 0/1728 [00:00<?, ?it/s]

Hp(qmult) 8.172556467354298


  0%|          | 0/1728 [00:00<?, ?it/s]

H(p) 7.114245407283306


In [168]:
# train model 2
model2 = build_model(generate_config(nvars=3, nvals=4, decoder_n_layer=16, tool_n_layer=8, seed=1349, hidden_dim=24, attention_mode="post_scale:albo", initializer_range=0.08, remove_ln=True))
model2.tool = model1.tool

n_unit = 1

params = list(model2.parameters())
optimizer = torch.optim.Adam(params, lr=1e-4)
unit_obs = 64
grad_acc_steps = n_unit
rngs = list(range(grad_acc_steps))
iter = 0
points2 = [(0, cross_entropy(model1, model2))]

  0%|          | 0/1728 [00:00<?, ?it/s]

In [169]:
steps = 30
for i in range(steps):
    cumloss = 0
    for i in range(grad_acc_steps):
        loss = cross_entropy_estimate(model1, model2, n=unit_obs, rng=rngs[i]) / grad_acc_steps
        # loss = cross_entropy_with_grad(model1, model2) / grad_acc_steps
        loss.backward()
        cumloss += loss.item()
    gnorm = sum(p.grad.norm()**2 for p in model2.parameters() if p.grad is not None) ** 0.5
    optimizer.step()
    optimizer.zero_grad()
    iter += 1
    if iter % 1 == 0:
        points2.append((iter, cross_entropy(model1, model2)))
    print(iter, cumloss, gnorm.item())

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/1728 [00:00<?, ?it/s]

1 8.043940544128418 28.15952491760254


  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/1728 [00:00<?, ?it/s]

...

  0%|          | 0/1728 [00:00<?, ?it/s]

29 6.1170549392700195 5.7506103515625


  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/1728 [00:00<?, ?it/s]

30 6.100719928741455 3.5992584228515625


In [126]:
points2

[(0, 8.112020283937454),
 (5, 7.3255085200071335),
 (10, 7.238180920481682),
 (15, 7.217939525842667),
 (20, 7.209257982671261),
 (25, 7.201131984591484),
 (30, 7.1905117854475975),
 (35, 7.181956596672535),
 (40, 7.178339675068855),
 (45, 7.176363982260227),
 (50, 7.176593787968159),
 (55, 7.17917013168335),
 (60, 7.182565651834011),
 (65, 7.18831180781126),
 (70, 7.194659382104874),
 (75, 7.199209943413734),
 (80, 7.197596035897732),
 (85, 7.196786507964134),
 (90, 7.203194804489613),
 (95, 7.206332750618458),
 (100, 7.21816473454237)]

In [132]:
# train model 3
model3 = build_model(generate_config(nvars=3, nvals=4, decoder_n_layer=16, tool_n_layer=8, seed=1349, hidden_dim=24, attention_mode="post_scale:mult", initializer_range=0.08, remove_ln=True))
model3.tool = model1.tool

n_unit = 10

params = list(model3.parameters())
optimizer = torch.optim.Adam(params, lr=1e-4)
unit_obs = 64
grad_acc_steps = n_unit
rngs = list(range(grad_acc_steps))
iter = 0
points3 = [(0, cross_entropy(model1, model3))]

  0%|          | 0/1728 [00:00<?, ?it/s]

In [133]:
steps = 100
for i in range(steps):
    cumloss = 0
    for i in range(grad_acc_steps):
        loss = cross_entropy_estimate(model1, model3, n=unit_obs, rng=rngs[i]) / grad_acc_steps
        # loss = cross_entropy_with_grad(model1, model2) / grad_acc_steps
        loss.backward()
        cumloss += loss.item()
    gnorm = sum(p.grad.norm()**2 for p in model3.parameters() if p.grad is not None) ** 0.5
    optimizer.step()
    optimizer.zero_grad()
    iter += 1
    if iter % 5 == 0:
        points3.append((iter, cross_entropy(model1, model3)))
    print(iter, cumloss, gnorm.item())

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

...

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/1728 [00:00<?, ?it/s]

100 6.573513627052307 27.64168357849121


In [139]:
points3

[(0, 8.281195513904095),
 (5, 7.4109926745295525),
 (10, 7.239325255155563),
 (15, 7.2092429250478745),
 (20, 7.1945459842681885),
 (25, 7.189999528229237),
 (30, 7.186420723795891),
 (35, 7.181824192404747),
 (40, 7.179786153137684),
 (45, 7.183007314801216),
 (50, 7.188995063304901),
 (55, 7.195444844663143),
 (60, 7.198489315807819),
 (65, 7.206330478191376),
 (70, 7.214569836854935),
 (75, 7.217803359031677),
 (80, 7.218523159623146),
 (85, 7.224212519824505),
 (90, 7.23375640809536),
 (95, 7.2563979253172874),
 (100, 7.26538348197937)]

In [135]:
points2

[(0, 8.112020283937454),
 (5, 7.3255085200071335),
 (10, 7.238180920481682),
 (15, 7.217939525842667),
 (20, 7.209257982671261),
 (25, 7.201131984591484),
 (30, 7.1905117854475975),
 (35, 7.181956596672535),
 (40, 7.178339675068855),
 (45, 7.176363982260227),
 (50, 7.176593787968159),
 (55, 7.17917013168335),
 (60, 7.182565651834011),
 (65, 7.18831180781126),
 (70, 7.194659382104874),
 (75, 7.199209943413734),
 (80, 7.197596035897732),
 (85, 7.196786507964134),
 (90, 7.203194804489613),
 (95, 7.206332750618458),
 (100, 7.21816473454237)]

In [163]:
# train model 4
model4 = build_model(generate_config(nvars=3, nvals=4, decoder_n_layer=16, tool_n_layer=8, seed=1349, hidden_dim=24, attention_mode="post_scale:mult", initializer_range=0.08, remove_ln=True), decoder_only=True)

n_unit = 1

params = list(model4.parameters())
optimizer = torch.optim.Adam(params, lr=1e-4)
unit_obs = 64
grad_acc_steps = n_unit
rngs = list(range(grad_acc_steps))
iter = 0
points4 = [(0, cross_entropy(model1, model4))]

  0%|          | 0/1728 [00:00<?, ?it/s]

In [164]:
steps = 30
for i in range(steps):
    cumloss = 0
    for i in range(grad_acc_steps):
        loss = cross_entropy_estimate(model1, model4, n=unit_obs, rng=rngs[i]) / grad_acc_steps
        # loss = cross_entropy_with_grad(model1, model2) / grad_acc_steps
        loss.backward()
        cumloss += loss.item()
    gnorm = sum(p.grad.norm()**2 for p in model4.parameters() if p.grad is not None) ** 0.5
    optimizer.step()
    optimizer.zero_grad()
    iter += 1
    if iter % 1 == 0:
        points4.append((iter, cross_entropy(model1, model4)))
    print(iter, cumloss, gnorm.item())

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/1728 [00:00<?, ?it/s]

1 7.305159568786621 16.526578903198242


  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/1728 [00:00<?, ?it/s]

...

  0%|          | 0/1728 [00:00<?, ?it/s]

29 6.187668323516846 5.3842244148254395


  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/1728 [00:00<?, ?it/s]

30 6.15560245513916 5.694852828979492


In [160]:
points2

[(0, 8.112020283937454),
 (5, 7.3255085200071335),
 (10, 7.238180920481682),
 (15, 7.217939525842667),
 (20, 7.209257982671261),
 (25, 7.201131984591484),
 (30, 7.1905117854475975),
 (35, 7.181956596672535),
 (40, 7.178339675068855),
 (45, 7.176363982260227),
 (50, 7.176593787968159),
 (55, 7.17917013168335),
 (60, 7.182565651834011),
 (65, 7.18831180781126),
 (70, 7.194659382104874),
 (75, 7.199209943413734),
 (80, 7.197596035897732),
 (85, 7.196786507964134),
 (90, 7.203194804489613),
 (95, 7.206332750618458),
 (100, 7.21816473454237)]

In [161]:
points3

[(0, 8.281195513904095),
 (5, 7.4109926745295525),
 (10, 7.239325255155563),
 (15, 7.2092429250478745),
 (20, 7.1945459842681885),
 (25, 7.189999528229237),
 (30, 7.186420723795891),
 (35, 7.181824192404747),
 (40, 7.179786153137684),
 (45, 7.183007314801216),
 (50, 7.188995063304901),
 (55, 7.195444844663143),
 (60, 7.198489315807819),
 (65, 7.206330478191376),
 (70, 7.214569836854935),
 (75, 7.217803359031677),
 (80, 7.218523159623146),
 (85, 7.224212519824505),
 (90, 7.23375640809536),
 (95, 7.2563979253172874),
 (100, 7.26538348197937)]

In [165]:
points4

[(0, 7.752054892480373),
 (1, 7.538040228188038),
 (2, 7.4261064156889915),
 (3, 7.356108255684376),
 (4, 7.316620022058487),
 (5, 7.299174375832081),
 (6, 7.291012465953827),
 (7, 7.283932328224182),
 (8, 7.277357518672943),
 (9, 7.273885354399681),
 (10, 7.275193408131599),
 (11, 7.281243585050106),
 (12, 7.2914049327373505),
 (13, 7.305211111903191),
 (14, 7.321758437901735),
 (15, 7.33851020783186),
 (16, 7.351692195981741),
 (17, 7.358934663236141),
 (18, 7.361220322549343),
 (19, 7.362230636179447),
 (20, 7.366258572787046),
 (21, 7.376065459102392),
 (22, 7.391542762517929),
 (23, 7.40978941321373),
 (24, 7.427623376250267),
 (25, 7.444806516170502),
 (26, 7.462954610586166),
 (27, 7.481636438518763),
 (28, 7.5002848990261555),
 (29, 7.5246289521455765),
 (30, 7.553483560681343)]

In [170]:
points2

[(0, 8.297052949666977),
 (1, 7.990744084119797),
 (2, 7.814562998712063),
 (3, 7.729019641876221),
 (4, 7.685854502022266),
 (5, 7.626898258924484),
 (6, 7.571107842028141),
 (7, 7.515889547765255),
 (8, 7.464568272233009),
 (9, 7.419832982122898),
 (10, 7.381143465638161),
 (11, 7.355037614703178),
 (12, 7.342437036335468),
 (13, 7.341261222958565),
 (14, 7.348810061812401),
 (15, 7.363933265209198),
 (16, 7.386332347989082),
 (17, 7.409938246011734),
 (18, 7.435428790748119),
 (19, 7.461574018001556),
 (20, 7.487233005464077),
 (21, 7.511971600353718),
 (22, 7.5345064625144005),
 (23, 7.553006559610367),
 (24, 7.5672366842627525),
 (25, 7.576402977108955),
 (26, 7.584069214761257),
 (27, 7.591229408979416),
 (28, 7.598849296569824),
 (29, 7.608126141130924),
 (30, 7.619076170027256)]